In [2]:
!pip install bert-score gpytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import pandas as pd

from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from typing import List


from gpytorch.models import ExactGP
from gpytorch.means import ZeroMean
from gpytorch.kernels import Kernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.distributions import MultivariateNormal
from gpytorch.mlls import ExactMarginalLogLikelihood

from bert_score import score as bert_score

import gc

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [5]:
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 20.6 MB/s eta 0:00:00


In [6]:
# import os

# path = os.listdir('/content/LLM-Human Classification Data.csv')[0]
# print("Input path:", path)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import pandas as pd

from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from typing import List

In [8]:
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"GPU Name: {torch.cuda.get_device_name(0)}")
        print(f"Total memory: {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2:.2f} MB")
        print(f"Allocated memory: {torch.cuda.memory_allocated(0) / 1024 ** 2:.2f} MB")
        print(f"Reserved memory: {torch.cuda.memory_reserved(0) / 1024 ** 2:.2f} MB")
        print(f"Free (unreserved) memory: {(torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)) / 1024 ** 2:.2f} MB")
    else:
        print("CUDA not available.")

print_gpu_memory()

GPU Name: Tesla T4
Total memory: 15095.06 MB
Allocated memory: 0.00 MB
Reserved memory: 0.00 MB
Free (unreserved) memory: 0.00 MB


In [9]:
dataset = pd.read_csv('/content/LLM-Human Classification Data.csv')

dataset.shape

(29145, 2)

In [10]:
llm_dataset = dataset[dataset['generated'] == 1]
human_dataset = dataset[dataset['generated'] == 0]
print(llm_dataset.shape, human_dataset.shape)

llm_dataset = llm_dataset[:int((llm_dataset.shape[0])*0.05)]
human_dataset = human_dataset[:int((human_dataset.shape[0])*0.03)]
print(llm_dataset.shape, human_dataset.shape)

smaller_dataset = pd.concat([llm_dataset, human_dataset])
smaller_dataset = smaller_dataset.sample(frac=1)
smaller_dataset.shape

(11637, 2) (17508, 2)
(581, 2) (525, 2)


(1106, 2)

In [11]:
print(smaller_dataset['generated'].value_counts())

generated
1    581
0    525
Name: count, dtype: int64


In [12]:
smaller_dataset.head()

,text,generated
364,"[Your Name]\n[Your Address]\n[City, State, Zip...",1
1244,"Dear Senator, I believe that people like Richa...",0
139,Advantages of Limiting Car Usage\n\nLimiting c...,1
312,"[Your Name]\n[Your Address]\n[City, State, Zip...",1
886,"Cars thery're every where, and people use cars...",0


In [13]:
class PerturbationGenerator:
    def __init__(self, model_name: str = "t5-3b"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.tokenizer.model_max_length = 512
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def perturb(self, text: str, num_samples: int = 9) -> List[str]:
        inputs = self.tokenizer(
            f"paraphrase: {text}", return_tensors="pt", max_length=512, truncation=True
        ).to(self.device)

        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            num_return_sequences=num_samples,
            do_sample=True,
            max_length=512,
        )

        return [text] + [self.tokenizer.decode(out, skip_special_tokens=True) for out in outputs]





class BertScoreKernel(Kernel):
    def __init__(self, sim_matrix: torch.Tensor):
        super().__init__()
        self.sim_matrix = sim_matrix
        self.register_parameter(
            name="raw_alpha",
            parameter = nn.Parameter(torch.tensor(0.0)))
        self.register_parameter(
            name="raw_beta",
            parameter = nn.Parameter(torch.tensor(0.0)))

    @property
    def alpha(self):
        return F.softplus(self.raw_alpha)

    @property
    def beta(self):
        return F.softplus(self.raw_beta)

    def forward(self, x1: torch.Tensor, x2: torch.Tensor, **kwargs) -> torch.Tensor:
        x1 = x1.long().squeeze()
        x2 = x2.long().squeeze()

        x1_idx = x1.unsqueeze(-1)
        x2_idx = x2.unsqueeze(0)

        K = self.alpha * self.sim_matrix[x1_idx, x2_idx] + self.beta
        return K





class SurrogateModel(ExactGP):
    def __init__(self, sim_matrix: torch.Tensor, train_indices: torch.Tensor, train_y: torch.Tensor):
        likelihood = GaussianLikelihood()
        super().__init__(train_indices, train_y, likelihood)
        self.mean_module = ZeroMean()    #  Can also try ConstantMean()
        self.covar_module = BertScoreKernel(sim_matrix)

    def forward(self, x: torch.Tensor) -> MultivariateNormal:
        x = x.long()
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

    def optimize_hyperparameters(self, train_x, train_y, lr=0.01, iterations=50):   #   May tune this "iteration" to 100
        self.train()
        self.likelihood.train()

        optimizer = Adam([
            {'params': self.covar_module.parameters()},
            {'params': self.likelihood.parameters()}
        ], lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=iterations)

        mll = ExactMarginalLogLikelihood(self.likelihood, self)

        for _ in range(iterations):
            optimizer.zero_grad()
            output = self(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
            scheduler.step()





class BayesianDetector:
    def __init__(self, source_model, perturbation_model="t5-3b", num_total_samples=10, uncertainty_threshold=0.1):
        self.num_total_samples = num_total_samples
        self.source_model = source_model
        self.perturbator = PerturbationGenerator(perturbation_model)
        self.gp = None
        self.all_texts = []
        self.uncertainty_threshold = uncertainty_threshold

    def get_log_prob(self, texts: List[str]) -> torch.Tensor:
        return self.source_model.get_log_prob(texts)

    def detect(self, text: str, max_queries: int = 5) -> float:
        self.all_texts = self.perturbator.perturb(
            text,
            num_samples=self.num_total_samples-1
        )
        max_queries = min(max_queries, self.num_total_samples - 2)

        num_texts = len(self.all_texts)
        sim_matrix = torch.zeros((num_texts, num_texts))
        with tqdm(total=((num_texts+1)*(num_texts) // 2), desc="Precomputing BERTScore") as pbar:
            for i in range(num_texts):
                for j in range(i, num_texts):
                    if i == j:
                        sim_matrix[i, j] = 1.0
                    else:
                        _, _, F1 = bert_score(
                            [self.all_texts[i]], [self.all_texts[j]],
                            lang="en",
                            model_type="bert-base-uncased",
                            rescale_with_baseline=True,
                            verbose=False
                        )
                        sim_matrix[i, j] = sim_matrix[j, i] = F1.item()
                    pbar.update(1)

        train_indices = torch.tensor([0, 1])
        y_train = self.get_log_prob([self.all_texts[0], self.all_texts[1]])

        for _ in tqdm(range(max_queries - 2), desc="Active Sampling"):
            self.gp = SurrogateModel(sim_matrix, train_indices, y_train)
            self.gp.optimize_hyperparameters(train_indices, y_train)

            remaining_indices = torch.tensor(
                [i for i in range(num_texts) if i not in train_indices]
            )

            with torch.no_grad():
                self.gp.eval()
                pred = self.gp.likelihood(self.gp(remaining_indices))
                uncertainties = pred.variance


            #   Here we are selecting the pertubations with the highest uncertainity
            if torch.max(uncertainties) < self.uncertainty_threshold:
                break

            next_idx = remaining_indices[torch.argmax(uncertainties)].item()
            train_indices = torch.cat([train_indices, torch.tensor([next_idx])])
            y_train = torch.cat([y_train, self.get_log_prob([self.all_texts[next_idx]])])

        with torch.no_grad():
            self.gp.eval()
            all_preds = self.gp(torch.arange(num_texts))

        original_score = all_preds.mean[0]
        perturbed_scores = all_preds.mean[1:]
        perturbed_mean = perturbed_scores.mean()
        perturbed_std = perturbed_scores.std()

        if perturbed_std == 0:
            perturbed_std = 1e-8

        normalized_score = (original_score - perturbed_mean) / perturbed_std
        return normalized_score.item()



class SourceModel:
    def __init__(self, model_name="gpt2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()

    def get_log_prob(self, texts: List[str]) -> torch.Tensor:
        log_probs = []
        with torch.no_grad():
            for text in texts:
                inputs = self.tokenizer(text, return_tensors="pt", truncation=True).to(self.device)

                # Handle empty input
                if inputs["input_ids"].shape[1] == 0:
                    log_probs.append(0.0)
                    continue

                # Handle single-token edge case
                num_tokens = inputs["input_ids"].shape[1]
                if num_tokens == 1:
                    log_probs.append(0.0)
                    continue

                outputs = self.model(**inputs, labels=inputs["input_ids"])
                logits = outputs.logits
                labels = inputs["input_ids"]

                log_probs_seq = torch.log_softmax(logits, dim=-1)[:, :-1, :]
                log_probs_text = torch.gather(
                    log_probs_seq,
                    dim=2,
                    index=labels[:, 1:, None]
                ).squeeze(-1).sum(dim=1)

                normalized_log_prob = log_probs_text.item() / (num_tokens - 1)
                log_probs.append(normalized_log_prob)

        print(log_probs)
        return torch.tensor(log_probs)

In [14]:
smaller_dataset.columns

Index(['text', 'generated'], dtype='object')

In [ ]:
# Assuming smaller_dataset is a pandas DataFrame with a 'text' column
sentences = list(smaller_dataset['text'])
scores = []

source_model = SourceModel(model_name="gpt2")

detector = BayesianDetector(
    source_model,
    perturbation_model="t5-small",
    num_total_samples=10,
    uncertainty_threshold=1e-3
)

for i, sentence in enumerate(sentences):
    detection_score = detector.detect(sentence, max_queries=5)
    if i % 15 == 0:
        print(f"Sentence {i+1}: Detection score = {detection_score}")
    if i % 50 == 0:
      with open("/content/drive/My Drive/my_numbers.txt", "w") as f:
                for number in scores:
                    f.write(f"{number}\n")

    scores.append(detection_score)

    # Free up GPU memory every 10 steps
    if i % 10 == 0:
        torch.cuda.empty_cache()
        gc.collect()

# Final cleanup
torch.cuda.empty_cache()
gc.collect()

smaller_dataset['score'] = scores
final_path = f"/content/drive/My Drive/bert_score_dataset_scores_final.csv"
smaller_dataset.to_csv(final_path, index=False)
print(f"Final save to {final_path}")

Precomputing BERTScore: 100%|██████████| 55/55 [00:27<00:00,  2.00it/s]
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


[-2.3267002979700133, -2.6761278126337755]


Active Sampling:  67%|██████▋   | 2/3 [00:00<00:00,  5.14it/s]

[-3.9133140563964846]
[-4.846860139266305]


Active Sampling: 100%|██████████| 3/3 [00:00<00:00,  5.36it/s]


...

Active Sampling:  67%|██████▋   | 2/3 [00:00<00:00,  6.59it/s]

[-3.7513042449951173]


Active Sampling: 100%|██████████| 3/3 [00:00<00:00,  6.59it/s]

[-2.40698358286982]



Precomputing BERTScore:  40%|████      | 22/55 [00:09<00:12,  2.61it/s]

In [1]:
final_path = f"/content/smaller_dataset_scores_final.csv"
smaller_dataset['score'] = scores
smaller_dataset.to_csv(final_path, index=False)
print(f"Final save to {final_path}")

Final save to /content/smaller_dataset_scores_final.csv


In [ ]:
print_gpu_memory()

GPU Name: Tesla P100-PCIE-16GB
Total memory: 16269.25 MB
Allocated memory: 495.18 MB
Reserved memory: 504.00 MB
Free (unreserved) memory: 8.82 MB


In [ ]:
score_dataset = smaller_dataset.copy()
score_dataset["pred score"] = pred_scores
score_dataset.head()

,text,generated,pred score
182,Limiting car usage can have several advantages...,1,1.492491
751,"""America's love affair with it's vehicles seem...",0,0.919343
801,Reducing the usage of cars in today's world co...,0,0.953056
385,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1,0.772685
495,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1,1.132379
